In [1]:
import sys
sys.path.append("./PriorFlowCVAE")

In [2]:
from main import main
import argparse
import itertools
import pandas as pd
import shutil
import os
import glob
import os

In [3]:
lr = [1e-4] #1e-3
num_particles = [20] #10
num_epochs = [200] #50
z = [750] #, 250 makla
hidd1 = [1500]  # Around the current hidden_1=500, 400, 500, makla
hidd2 = [1000]  # Around the current hidden_2=500
random_mask = False

# Iterate over all combinations of hyperparameters
for lr, num_ep, num_part, z, hid1, hid2 in itertools.product(lr, num_epochs, num_particles, z, hidd1, hidd2):
    # Set up arguments
    args = argparse.Namespace(
        num_quadrant_inputs=[1, 2, 3],
        num_epochs=num_ep,
        early_stop_patience=3,
        learning_rate=lr,
        cuda=True,
        num_images=10,
        num_samples=10,
        num_particles=num_part,
        dataset='cifar10',
        z_dim=z,
        random_mask=random_mask,
        hidden_1=hid1,
        hidden_2=hid2,
        allow_baseline_reuse=False,
        use_conv=False
    )

    # Log the current configuration
    print(f"Running with lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}")

    # Run the main function
    try:
        main(args)
    except Exception as e:
        print(f"Error with configuration lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}: {e}")
        continue

    # Save the results
    if os.path.exists("results.csv"):
        output_file = f"tuning_results/results_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.csv"
        shutil.copy("results.csv", output_file)
        print(f"Results saved to {output_file}")
    else:
        print(f"No 'results.csv' file generated for lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}.")
    # Backup important files before starting
    if os.path.exists("cvae_plot_q1.png"):
        output_file = f"tuning_results/cvae_ploth_q1_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.png"
        shutil.copy("cvae_plot_q1.png", output_file)
        print("Backup created for 'cvae_plot_q1.png'.")
    if os.path.exists("cvae_plot_q2.png"):
        output_file = f"tuning_results/cvae_ploth_q2_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.png"
        shutil.copy("cvae_plot_q2.png", output_file)
        print("Backup created for 'cvae_plot_q2.png'.")
    if os.path.exists("cvae_plot_q3.png"):
        output_file = f"tuning_results/cvae_ploth_q3_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.png"
        shutil.copy("cvae_plot_q3.png", output_file)
        print("Backup created for 'cvae_plot_q3.png'.")


Running with lr=0.0001, num_epochs=[200], num_particles=[20]
Device:  cuda:0
Training with 1 quadrant as input...
Files already downloaded and verified
Files already downloaded and verified


CVAE Epoch 125 val  : 100%|██████████| 10/10 [00:01<00:00,  5.88it/s, early_stop_count=2, loss=-1697.54]


Files already downloaded and verified
Files already downloaded and verified


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..1.2582052946090698].


Files already downloaded and verified
Files already downloaded and verified


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..1.0741379261016846].


Files already downloaded and verified
Files already downloaded and verified


Generating predictions: 100%|██████████| 313/313 [00:36<00:00,  8.60it/s]

Df:                      3 quadrants
NN (baseline)       2546.707493
CVAE (Monte Carlo)  -592.441949
Columns:  ['1 quadrant', '2 quadrants', '3 quadrants']
Results saved to tuning_results/results_lr_0.0001_epochs_[200]_particles_[20].csv
Backup created for 'cvae_plot_q1.png'.
Backup created for 'cvae_plot_q2.png'.
Backup created for 'cvae_plot_q3.png'.


<Figure size 640x480 with 0 Axes>

In [4]:
# Combine all results for analysis
result_files = glob.glob("tuning_results/*.csv")
if result_files:
    combined_results = []

    for file in result_files:
        # Read each result file
        df = pd.read_csv(file)

        # Rename the first column if necessary
        if 'Unnamed: 0' in df.columns:
            df.rename(columns={'Unnamed: 0': 'Method'}, inplace=True)

        print(df)

        # Extract performance gaps and compute the aggregate gap
        performance_gap_row = df.loc[df['Method'] == 'Performance gap']
        aggregate_gap = performance_gap_row[['1 quadrant', '2 quadrants', '3 quadrants']].sum(axis=1).values[0]

        # Store results with the file name and aggregate gap
        combined_results.append({
            "file": file,
            "aggregate_gap": aggregate_gap
        })

    # Convert to a DataFrame for sorting and analysis
    combined_results_df = pd.DataFrame(combined_results)

    # Sort by aggregate gap in descending order (largest gap first)
    combined_results_df = combined_results_df.sort_values(by="aggregate_gap", ascending=False)

    # Save the combined results to a file
    combined_results_df.to_csv("tuning_results/combined_results.csv", index=False)
    print("All results combined and saved to 'tuning_results/combined_results.csv'.")

    # Display the best configuration
    print("Best configuration:")
    best_file = combined_results_df.iloc[0]['file']
    print(f"File: {best_file}, Aggregate Performance Gap: {combined_results_df.iloc[0]['aggregate_gap']}")

    # Optionally, load and display the details of the best result
    best_result = pd.read_csv(best_file)
    print("\nDetails of the best configuration:")
    print(best_result)
else:
    print("No results to combine.")


               Method    1 quadrant  2 quadrants  3 quadrants
0       NN (baseline)  15330.969677  6950.834092  2546.707493
1  CVAE (Monte Carlo)  -1728.758257 -1663.301821  -592.441949
2     Performance gap  17059.727934  8614.135913  3139.149442
               Method   1 quadrant  2 quadrants  3 quadrants
0       NN (baseline)  1492.618233   963.135504   478.029860
1  CVAE (Monte Carlo) -2772.922125 -1881.511512  -896.795618
2     Performance gap  4265.540358  2844.647016  1374.825478
               Method   1 quadrant  2 quadrants  3 quadrants
0       NN (baseline)  1496.599199   964.483810   478.570291
1  CVAE (Monte Carlo)  1397.227819   932.649323   497.036486
2     Performance gap    99.371380    31.834487   -18.466196
                                                file  aggregate_gap
0  tuning_results/results_lr_5e-05_epochs_[101]_p...     112.739671


KeyError: 'Method'